In [407]:
import numpy as np
from dotenv import load_dotenv
from dwave.system import DWaveSampler, EmbeddingComposite
from numpy import hamming
from openjij import SQASampler
from pyqubo import Array, Constraint, LogEncInteger, solve_qubo
import math
import os
import dimod

In [408]:
load_dotenv()

TOKEN = os.getenv("TOKEN")
endpoint = "https://cloud.dwavesys.com/sapi/"

In [409]:
W = 995
cost = [
94,
506,
416,
992,
649,
237,
457,
815,
446,
422,
791,
359,
667,
598,
7,
544,
334,
766,
994,
893,
633,
131,
428,
700,
617,
874,
720,
419,
794,
196,
997,
116,
908,
539,
707,
569,
537,
931,
726,
487,
772,
513,
81,
943,
58,
303,
764,
536,
724,
789,
479,
142,
339,
641,
196,
494,
66,
824,
208,
711,
800,
314,
289,
401,
466,
689,
833,
225,
244,
849,
113,
379,
361,
65,
486,
686,
286,
889,
24,
491,
891,
90,
181,
214,
17,
472,
418,
419,
356,
682,
306,
201,
385,
952,
500,
194,
737,
324,
992,
224,]
weight = [ 485,
326,
248,
421,
322,
795,
43,
845,
955,
252,
9,
901,
122,
94,
738,
574,
715,
882,
367,
984,
299,
433,
682,
72,
874,
138,
856,
145,
995,
529,
199,
277,
97,
719,
242,
107,
122,
70,
98,
600,
645,
267,
972,
895,
213,
748,
487,
923,
29,
674,
540,
554,
467,
46,
710,
553,
191,
724,
730,
988,
90,
340,
549,
196,
865,
678,
570,
936,
722,
651,
123,
431,
508,
585,
853,
642,
992,
725,
286,
812,
859,
663,
88,
179,
187,
619,
261,
846,
192,
261,
514,
886,
530,
849,
294,
799,
391,
330,
298,
790,]
opt = [0,
1,
0,
1,
0,
0,
1]
c = {}
w = {}
N = len(cost)
for i in range(N):
    c[i] = cost[i]
    w[i] = weight[i]

In [410]:
x = Array.create('x', shape=(N), vartype='BINARY')
y = LogEncInteger("y", (0, W))

In [411]:
key1 = max(c, key=lambda k: c[k])
B = 1
A = B * c[key1] * 3

In [412]:
HA = Constraint(
    A * (W - sum(w[a] * x[a] for a in range(N)) - y)**2, label='HA'
)

HB = - B * sum(c[a] * x[a] for a in range(N))

In [413]:
H = HA + HB
model = H.compile()
Q, offset = model.to_qubo()

In [414]:
dw_sampler = DWaveSampler(solver='DW_2000Q_6',
                          token=TOKEN, endpoint=endpoint)
sampler = EmbeddingComposite(dw_sampler)
sampleset = sampler.sample_qubo(Q, num_reads=30)

ValueError: no embedding found

In [ ]:
sampleset.record

rec.array([([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0], -2.34467568e+09, 1, 0.08333333),
           ([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], -1.28769194e+09, 1, 0.08333333),
           ([0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0],  3.31006968e+09, 1, 0.05      ),
           ([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0],  1.60816887e+09, 1, 0.1       ),
           ([0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
decoded_sample = model.decode_sample(sampleset.first.sample, vartype="BINARY")

In [ ]:
cost = 0
weight = 0
for k in range(N):
    if decoded_sample.array('x', k) != 0:
        cost += c[k]
        weight += w[k]

In [ ]:
sol_y = sum(2**k * v for k, v in [(elem, decoded_sample.array('y', elem))
            for elem in range(math.ceil(math.log2(W)))])

In [ ]:
print("スラック変数Y = {}".format(sol_y))
print()
print("broken")
print(decoded_sample.constraints(only_broken=True))
print("cost : "+str(cost))
print("weight : "+str(weight))

スラック変数Y = 8

broken
{'HA': (False, 6328956.0)}
cost : 3389
weight : 941


In [ ]:
init = sampleset.samples()[-1]

最悪値

In [ ]:
sol_y = sum(2**k * v for k, v in [(elem, decoded_sample.array('y', elem))
            for elem in range(math.ceil(math.log2(W)))])

In [ ]:
t = (0, 5, 15, 20)
s = (1.0, 0.4, 0.4, 1.0)

schedule = list(zip(t, s))

In [ ]:
dw_sampler = DWaveSampler(solver='DW_2000Q_6',
                          token=TOKEN, endpoint=endpoint)
sampler = EmbeddingComposite(dw_sampler)
sampleset = sampler.sample_qubo(
    Q, num_reads=30, anneal_schedule=schedule, initial_state=init)

In [ ]:
sampleset.record

rec.array([([0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0],  2.61357566e+10, 1, 0.2       ),
           ([0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0],  5.22138805e+10, 1, 0.23333333),
           ([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0],  6.37582548e+10, 1, 0.23333333),
           ([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0],  3.61581349e+08, 1, 0.18333333),
           ([0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0

In [ ]:
decoded_sample = model.decode_sample(sampleset.first.sample, vartype="BINARY")

In [ ]:
cost = 0
weight = 0
for k in range(N):
    if decoded_sample.array('x', k) != 0:
        cost += c[k]
        weight += w[k]

In [ ]:
sol_y = sum(2**k * v for k, v in [(elem, decoded_sample.array('y', elem))
            for elem in range(math.ceil(math.log2(W)))])

In [ ]:
print("スラック変数Y = {}".format(sol_y))
print()
print("broken")
print(decoded_sample.constraints(only_broken=True))
print("cost : "+str(cost))
print("weight : "+str(weight))

スラック変数Y = 62

broken
{'HA': (False, 547903344.0)}
cost : 3883
weight : 1361
